In [1]:
import finnhub
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import re
import time

C:\Users\jordi\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jordi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\jordi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
class trade_idea:
    def __init__(self, symbol, time_frame, position, time_posted, user):
        self.symbol = symbol
        self.time_frame = time_frame
        self.position = position
        self.time_posted = time_posted
        self.user = user

def get_trade_ideas(begin_page, end_page):
    trade_ideas = []
    for i in range(begin_page, end_page):
        trade_ideas.extend(store_all_data("https://www.tradingview.com/markets/stocks-usa/ideas/page-%d/" % i))
    print(len(trade_ideas))
    return trade_ideas

def store_all_data(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    data = soup.find_all("div", class_="tv-widget-idea js-userlink-popup-anchor")
    trade_ideas = []
    print(len(data))
    for i in data:
        symbol = i.find('a', class_="tv-widget-idea__symbol apply-overflow-tooltip")
        time_frame = i.find_all('span', class_="tv-widget-idea__timeframe")[1]
        position = i.find('span', class_="content-yHuWj4ze badge-idea-content-Naj79Gc7")
        time_posted = i.find('span', class_="tv-card-stats__time")
        user = i.find('span', class_="tv-card-user-info__name")
        if(symbol != None and time_frame != None and position != None and time_posted != None and user != None):
            trade_ideas.append(trade_idea(symbol.text, time_frame.text, position.text, time_posted['data-timestamp'], user.text))          
    return trade_ideas

def calculate_profit(stock, profit_exit, loss_exit):
    buy = 100
    initial_price = stock['Open'][0]
    exit1 = profit_exit
    exit2 = loss_exit
    for i in stock['Open']:
        if i / initial_price > exit1:
            print("initial_price: ", initial_price, "current price: ", i)
            return (buy * (i / initial_price)) - buy
        if i / initial_price < exit2:
            print("initial_price: ", initial_price, "current price: ", i)
            return (buy * (i / initial_price)) - buy
    return 0

def calculate_total_long_profit(trade_ideas, profit_exit_margin, exit_trade_margin):
    sum = 0
    for i in trade_ideas:
        stock = get_stock(i.symbol, i.time_posted, time.time())
        if i.position == "Long":
            profit = calculate_profit(stock, 1+profit_exit_margin, 1-exit_trade_margin)
            sum += profit
            print(i.symbol, i.time_frame, i.position, profit)
            
        if i.position == "Short":
            profit = -calculate_profit(stock, 1+exit_trade_margin, 1-profit_exit_margin)
            sum += profit
            print(i.symbol, i.time_frame, i.position, profit)
    print("TOTAL PROFIT: ", sum)

def get_stock(stock_symbol, date_begin, date_end):
    data = finnhub_client.stock_candles(stock_symbol, 'D', int(float(date_begin)), int(float(date_end)))
    df = pd.DataFrame(data)
    df['t'] = df['t'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    stock = df[['t','o','h','l','c','v']].set_index('t')
    stock.rename(columns={'o': 'Open', 'h': 'High', 'l':'Low','c':'Close','v':'Volume'}, inplace=True)
    return stock        

def print_trade_ideas():
    for i in trade_ideas:
        print(i.symbol, i.time_frame, i.position, i.time_posted, i.user)


In [5]:
finnhub_client = finnhub.Client(api_key="cb40dtaad3i8tak19g00")

In [6]:
trade_ideas = get_trade_ideas(148,151)

18
18
18
36


In [7]:
calculate_total_long_profit(trade_ideas, 0.010, 0.005)

initial_price:  394.16 current price:  398.92
SPY 1D Long 1.2076314187132056
initial_price:  15.77 current price:  15.59
SH 1D Long -1.141407736207995
initial_price:  408.33 current price:  414.0
ROP 1D Short -1.3885827639409314
initial_price:  13.61 current price:  13.81
EOLS 1D Long 1.4695077149155225
initial_price:  765.32 current price:  828.66
TSLA 1W Short -8.27627659018448
initial_price:  71.32 current price:  72.5
SCI 1D Long 1.6545148625911423
initial_price:  11.34 current price:  11.56
PBR 1D Long 1.9400352733686077
initial_price:  320.51 current price:  327.11
GS 1D Short -2.0592181211194713
initial_price:  178.85 current price:  170.19
NVDA 60 Short 4.842046407604144
initial_price:  123.2 current price:  125.01
AMZN 15 Short -1.4691558441558499
initial_price:  394.16 current price:  398.92
SPY 1D Long 1.2076314187132056
initial_price:  765.32 current price:  828.66
TSLA 15 Long 8.27627659018448
initial_price:  154.5 current price:  155.39
AAPL 180 Short -0.5760517799352698


In [54]:
stock = get_stock("SPY", 1657344370.0, time.time())

In [56]:
original_price = stock["Open"][0]

In [57]:
original_price

385.85

In [62]:
stock['% change'] = stock.apply(lambda x: 1 - (x['Open'] / original_price), axis = 1)

In [63]:
stock

,Open,High,Low,Close,Volume,% change
t,,,,,,
2022-07-11 02:00:00,385.85,386.8700,383.500,384.23,58366945,0.000000
2022-07-12 02:00:00,383.65,386.1600,378.990,380.83,62219178,0.005702
2022-07-13 02:00:00,375.10,381.9200,374.658,378.83,84224649,0.027861
2022-07-14 02:00:00,373.61,379.0498,371.040,377.91,89704819,0.031722
2022-07-15 02:00:00,382.55,385.2500,380.540,385.13,79060383,0.008553
2022-07-18 02:00:00,388.38,389.0900,380.660,381.95,63203626,-0.006557
2022-07-19 02:00:00,386.08,392.8700,385.390,392.27,78505972,-0.000596
2022-07-20 02:00:00,392.47,396.2600,391.030,394.77,71843769,-0.017157
2022-07-21 02:00:00,394.16,398.8400,391.630,398.79,64903856,-0.021537
